In [1]:
import pandas as pd
import numpy as np

import jieba
import jieba.posseg as pseg
import jieba.analyse

import sys
from optparse import OptionParser
import pprint
from prettytable import PrettyTable

In [2]:
df = pd.read_csv(open('../data/df_small.csv'))
df = df[df['text'].str.contains("https|Britain")== False]
print('有{}筆對話'.format(len(df)))

有91340筆對話


## groupby

In [3]:
grouped_data = df.groupby('from')['text'].apply(lambda x: '/'.join(x)).reset_index()
l = len(grouped_data)
print('有{}個使用者'.format(l))

有1019個使用者


## 先用前1000筆資料做測試

In [75]:
test = grouped_data.head(1000)
test_len = len(test)
test.head()

,from,text
0,1000970273334199,找不到阿/安安/你有打LOL嗎/好吧，可以帶我嗎/安安/凸/Sor/嗯/約哪/有啊/你先約的...
1,1001070263323689,!?/唐寶寶/你那麼愛慕他ㄇ/嗨/先問問您期望本人的性別是?/了解/我是男的/抱歉沒有/請問...
2,1001967239901904,早/在床上發懶/爆/斷/魂/你在暗諷綠藻嘛/壞/話說我好餓ㄛ/窩不要這個qq/ㄋ早餐吃什麼/...
3,1003172849781848,甲/你要做我的0？/人生勝利組是不會來玩這個的/你一定不是c銘/也不是業王/葉你妹/被盜/其...
4,1003929993039131,吃噓/肥宅早/我們不是場外人/場外ㄈㄓ/我沒用了/安安/是喔/場外ㄈㄓ嗎/豪/斷/欸/在打手...


In [5]:
for i in range(len(test)):
    test.iloc[i][1] = test.iloc[i][1].split('/')
test.head()

,from,text
0,1000970273334199,"[找不到阿, 安安, 你有打LOL嗎, 好吧，可以帶我嗎, 安安, 凸, Sor, 嗯, 約..."
1,1001070263323689,"[!?, 唐寶寶, 你那麼愛慕他ㄇ, 嗨, 先問問您期望本人的性別是?, 了解, 我是男的,..."
2,1001967239901904,"[早, 在床上發懶, 爆, 斷, 魂, 你在暗諷綠藻嘛, 壞, 話說我好餓ㄛ, 窩不要這個q..."
3,1003172849781848,"[甲, 你要做我的0？, 人生勝利組是不會來玩這個的, 你一定不是c銘, 也不是業王, 葉你..."
4,1003929993039131,"[吃噓, 肥宅早, 我們不是場外人, 場外ㄈㄓ, 我沒用了, 安安, 是喔, 場外ㄈㄓ嗎, ..."


In [6]:
print('id為 {} 的使用者傳送的訊息{}'.format(test.iloc[0][0],test.iloc[0][1]))

id為 1000970273334199 的使用者傳送的訊息['找不到阿', '安安', '你有打LOL嗎', '好吧，可以帶我嗎', '安安', '凸', 'Sor', '嗯', '約哪', '有啊', '你先約的啊', '喔', '輸入0088', '我懶的打了', '皮米屌', '這你嗎？', '魏癢癢', '哪裡人?', '住哪裡啦？', '超癢', '哪裡人?', '好遠', '其實還好', '新竹', '龍潭?', 'ㄏㄅㄨㄐㄅㄌㄗㄋ', '三小', '幹又ㄋ', '掰掰', '凹?', '凸sor']


In [7]:
jieba.set_dictionary('jieba_txt/dict.txt.big.txt')
stopwordset = set()
with open('jieba_txt/stop_words.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
stopwordset

{'10',
 '20',
 '2016',
 '?',
 '他',
 '他們',
 '你',
 '台灣',
 '她',
 '對於',
 '成為',
 '我',
 '我們',
 '沒有',
 '王隆甫',
 '？'}

In [28]:
chars = set(' abcdefghijklmnopqrstuvwxyz0123456789()?!$,:.。，-_&=/􀆿\r\n')
for c in chars:
    stopwordset.add(c)

In [9]:
# stopwordset

## gensim model

In [29]:
from gensim import models
import gensim

In [30]:
sentences = models.word2vec.LineSentence("corpusSegDone.txt")
model = models.word2vec.Word2Vec(sentences, size=1, window=5, min_count=5, workers=1)
model.wv.vocab

{'白痴': <gensim.models.keyedvectors.Vocab at 0x7f6dd88a92e8>,
 '腐女': <gensim.models.keyedvectors.Vocab at 0x7f6dd859d160>,
 '那時候': <gensim.models.keyedvectors.Vocab at 0x7f6dd85c3828>,
 '誌': <gensim.models.keyedvectors.Vocab at 0x7f6dd8530cc0>,
 '讓': <gensim.models.keyedvectors.Vocab at 0x7f6dd85decf8>,
 '爛': <gensim.models.keyedvectors.Vocab at 0x7f6dd88a1cf8>,
 'Xd': <gensim.models.keyedvectors.Vocab at 0x7f6dd85515c0>,
 '永遠': <gensim.models.keyedvectors.Vocab at 0x7f6dd859d1d0>,
 '進': <gensim.models.keyedvectors.Vocab at 0x7f6dd8587550>,
 '南': <gensim.models.keyedvectors.Vocab at 0x7f6dd85a2358>,
 '刷': <gensim.models.keyedvectors.Vocab at 0x7f6dd8574668>,
 '揍': <gensim.models.keyedvectors.Vocab at 0x7f6dd882cc50>,
 '葉王': <gensim.models.keyedvectors.Vocab at 0x7f6dd85430f0>,
 '吉娃娃': <gensim.models.keyedvectors.Vocab at 0x7f6dd88a9550>,
 '改名': <gensim.models.keyedvectors.Vocab at 0x7f6dd85c36a0>,
 '承認': <gensim.models.keyedvectors.Vocab at 0x7f6dd85948d0>,
 '版主': <gensim.models.keyedve

## pick user top 10 word ( frequency-based )

In [93]:
top10_vector = np.zeros(shape=(l,10))
# top10_vector

In [94]:
import operator
for i in range(l):
    og_dict = dict()
#     print('\n user'+str(i+1))
#     if (len(test.iloc[i][1]))<10:
#         continue
    for j in range(len(grouped_data.iloc[i][1])):
        seglist = jieba.cut((grouped_data.iloc[i][1])[j])
        for word in seglist:
            if word not in stopwordset:
                if not word in og_dict.keys():
                    if word == " ":
                        continue
                    og_dict[word] = 1
                else:
                    og_dict[word] = int(og_dict[word]) + 1
    a = sorted(og_dict.items(), key=operator.itemgetter(1),reverse=True)
#     print(a)
    
    row = np.zeros(shape=(1,10))
    
    if len(a) >= 10:
        for x in range(10):
            if a[x][0] in model:
                show = float(model[a[x][0]])
#                 print(show)
                row[0][x] = show
    else:
        for x in range(len(a)):
#             print(a[x]) 
            if a[x][0] in model:
                show = float(model[a[x][0]])
#                 print(show)
                row[0][x] = show
#     print(row)
    top10_vector[i] = row

In [95]:
top10_vector.shape

(1019, 10)

In [74]:
# top10_vector

## 分群

In [99]:
from sklearn import cluster, datasets

# KMeans 演算法
kmeans_fit = cluster.KMeans(n_clusters = 3,random_state=1170).fit(top10_vector)

# 印出分群結果
cluster_labels = kmeans_fit.labels_
print("分群結果 前10：")
print(cluster_labels[:10])
print("---")




分群結果 前10：
[2 0 0 0 0 0 0 0 0 1]
---


## 分群結果數量

In [98]:
unique, counts = np.unique(cluster_labels, return_counts=True)
dict(zip(unique, counts))


{0: 595, 1: 110, 2: 314}